# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
import sklearn
sklearn.__version__

'0.20.0'

In [70]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [72]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
df_mean = df.groupby(['Pclass'])['Fare'].mean().reset_index()
df_median = df.groupby(['Pclass'])['Fare'].median().reset_index()
df_mode = df.groupby(['Pclass'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()

In [73]:
df1 = df.merge(df_mean, on='Pclass', how='left',suffixes=('_reg','_mean'))
df1 = df1.merge(df_median, on='Pclass', how='left',suffixes=('_x','_median'))
df1 = df1.merge(df_mode, on='Pclass', how='left',suffixes=('_median','_mode'))

In [74]:
df1.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare_reg,Cabin,Embarked,Fare_mean,Fare_median,Fare_mode
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,13.675550,8.0500,8.05
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.154687,60.2875,26.55
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,13.675550,8.0500,8.05
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,84.154687,60.2875,26.55
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,13.675550,8.0500,8.05


In [77]:
num_features_df = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features_df.append(feature)

df = df[num_features_df]
df = df.fillna(-1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
Pclass    891 non-null int64
Age       891 non-null float64
SibSp     891 non-null int64
Parch     891 non-null int64
Fare      891 non-null float64
dtypes: float64(2), int64(3)
memory usage: 34.9 KB


In [78]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df1.dtypes, df1.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df1 = df1[num_features]
df1 = df1.fillna(-1)
MMEncoder = MinMaxScaler()
df1.head()

8 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare_reg', 'Fare_mean', 'Fare_median', 'Fare_mode']



,Pclass,Age,SibSp,Parch,Fare_reg,Fare_mean,Fare_median,Fare_mode
0,3,22.0,1,0,7.2500,13.675550,8.0500,8.05
1,1,38.0,1,0,71.2833,84.154687,60.2875,26.55
2,3,26.0,0,0,7.9250,13.675550,8.0500,8.05
3,1,35.0,1,0,53.1000,84.154687,60.2875,26.55
4,3,35.0,0,0,8.0500,13.675550,8.0500,8.05


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [79]:
# 原始特徵 + 邏輯斯迴歸
estimator = LogisticRegression()
train_X_origin = df
print(cross_val_score(estimator, train_X_origin, train_Y, cv=5).mean())

0.6960299128976762


In [81]:
# 新特徵 + 邏輯斯迴歸
estimator = LogisticRegression()
train_X_new = df1[['Pclass', 'Age', 'SibSp', 'Parch','Fare_mean']]
print(cross_val_score(estimator, train_X_new, train_Y, cv=5).mean())

0.6870221757884549


In [82]:
estimator = LogisticRegression()
train_X_new = df1[['Pclass', 'Age', 'SibSp', 'Parch','Fare_median']]
print(cross_val_score(estimator, train_X_new, train_Y, cv=5).mean())

0.6870221757884549


In [83]:
estimator = LogisticRegression()
train_X_new = df1[['Pclass', 'Age', 'SibSp', 'Parch','Fare_mode']]
print(cross_val_score(estimator, train_X_new, train_Y, cv=5).mean())

0.6915229058081337


In [ ]:
# in this case mode has the best score